# Système d’Aide à la Décision (SAD) – Diagnostic de tumeurs cérébrales
Module Machine Learning (ALIF83) – 2025–2026
Yann Renvoisé, Aïssa Mehenni, LSI2

## 0. Contexte et objectif
- Différence classification vs aide à la décision
- Objectifs du SAD (confiance, triage, actions, minimiser faux négatifs)

## 1. Setup
- Imports
- Seed
- Paramètres globaux (paths, image size, batch size)

In [ ]:
import sys
from pathlib import Path
import numpy as np

PROJECT_ROOT = Path.cwd().parent  # notebooks/ -> Projet_SAD/
sys.path.append(str(PROJECT_ROOT))

from src.preprocessing import get_default_config, list_images_by_class, compute_class_counts, load_image_cv2, normalize_0_1, make_sklearn_features
from src.models import train_logistic_regression
from src.calibration import calibrate_sklearn_classifier
from src.decision_engine import DecisionThresholds, generer_recommandation
from src.reporting import creer_rapport_decision

cfg = get_default_config(PROJECT_ROOT)

print("Project root:", PROJECT_ROOT)
print("Train dir:", cfg.train_dir)
print("Test dir:", cfg.test_dir)
print("Classes:", cfg.class_names)

## 2. Données
### 2.1 Chargement du dataset (Training/Testing)
- Listing des classes
- Comptage des images par classe
### 2.2 Visualisation
- Quelques exemples par classe
### 2.3 Prétraitements
- Redimensionnement
- Normalisation
- Augmentation (train uniquement)

In [ ]:
train_imgs = list_images_by_class(cfg.train_dir, cfg.class_names)
test_imgs  = list_images_by_class(cfg.test_dir, cfg.class_names)

print("Train counts:", compute_class_counts(train_imgs))
print("Test counts :", compute_class_counts(test_imgs))

# Vérif rapide: au moins 1 image par classe
for c in cfg.class_names:
    assert len(train_imgs[c]) > 0, f"Aucune image train pour {c}"
    assert len(test_imgs[c]) > 0, f"Aucune image test pour {c}"

## TEST Petit sample -> Patient P_00001

In [ ]:
def build_dataset(images_by_class, class_names, n_per_class: int, image_size):
    X_list = []
    y_list = []
    skipped = 0

    for idx, c in enumerate(class_names):
        paths = images_by_class[c][:n_per_class]
        for p in paths:
            try:
                img = load_image_cv2(p, image_size)
            except Exception as e:
                skipped += 1
                # Log minimal
                print(f"Skip: {p} ({type(e).__name__})")
                continue

            img = normalize_0_1(img)
            feat = make_sklearn_features(img)  # flatten
            X_list.append(feat)
            y_list.append(idx)

    if len(X_list) == 0:
        raise RuntimeError("Aucune image n'a pu être chargée (tout a été skip).")

    X = np.stack(X_list, axis=0)
    y = np.array(y_list, dtype=np.int64)
    print(f"Dataset construit: {len(y)} samples, skipped={skipped}")
    return X, y


X_train, y_train = build_dataset(train_imgs, cfg.class_names, n_per_class=200, image_size=cfg.image_size)
X_test, y_test   = build_dataset(test_imgs, cfg.class_names, n_per_class=80,  image_size=cfg.image_size)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split calibration
X_fit, X_cal, y_fit, y_cal = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

res = train_logistic_regression(X_fit, y_fit, max_iter=2000)
base_model = res.model

# Calibration (isotonic ou sigmoid)
calibrated = calibrate_sklearn_classifier(base_model, X_cal, y_cal, method="isotonic")

# Eval rapide
proba_test = calibrated.predict_proba(X_test)
y_pred = proba_test.argmax(axis=1)
acc = accuracy_score(y_test, y_pred)
print("Test accuracy (calibrated):", round(acc, 4))

In [ ]:
# Prendre un exemple dans le test (ici le 1er "notumor" si dispo, sinon le 1er tout court)
target_class = "notumor"
sample_path = test_imgs[target_class][0] if len(test_imgs[target_class]) > 0 else list(test_imgs.values())[0][0]

img = load_image_cv2(sample_path, cfg.image_size)
img = normalize_0_1(img)
feat = make_sklearn_features(img).reshape(1, -1)

proba = calibrated.predict_proba(feat)[0]

scores_by_class = {cfg.class_names[i]: float(proba[i]) for i in range(len(cfg.class_names))}
thresholds = DecisionThresholds()
decision = generer_recommandation(scores_by_class, thresholds)

print("Sample:", sample_path.name)
print("Scores:", {k: round(v, 4) for k, v in scores_by_class.items()})
print("Decision:", decision)

In [ ]:
patient_id = "P_00001"
report = creer_rapport_decision(
    patient_id=patient_id,
    scores_by_class=scores_by_class,
    decision=decision,
)

print(report)

## 3. Baselines : Régression Logistique calibrée
### 3.1 Extraction de features (baseline)
- Approche simple (ex: flatten / features classiques)
### 3.2 Entraînement RegLog multinomiale
### 3.3 Calibration (Platt ou Isotonic)
### 3.4 Analyse des scores
- Distribution max_prob
- Cas incertains (max_prob < 0.7)

## 4. MLP probabiliste et limites
### 4.1 Préparation des features pour MLP
### 4.2 Entraînement MLP
### 4.3 Analyse confiance / erreurs
- Où le modèle se trompe
- Tranches de confiance

## 5. CNN calibré pour la décision
### 5.1 Dataloaders image (tensor)
### 5.2 Architecture CNN
### 5.3 Entraînement
### 5.4 Temperature Scaling (calibration)
### 5.5 Sauvegarde d’activations (pour analyse)

## 6. Moteur de décision clinique (règles & seuils)
- Implémentation des seuils
- Gestion asymétrique des faux négatifs (notumor < 0.95)

## 7. Tableau de bord : génération de rapports
- Générer des rapports textuels pour 20 patients (échantillon)
- Afficher quelques rapports dans le notebook

## 8. Analyse de performance orientée "métier"
### 8.1 Couverture automatique
### 8.2 Accuracy par tranche de confiance
- Vérifier accuracy > 95% quand confiance > 0.85
### 8.3 Analyse coût-bénéfice
- Coût = (FN*1000) + (FP*100) + (Revision*50)

## 9. Incertitude (MC Dropout)
- Estimation sur quelques images
- Comparaison avec confiance softmax

## 10. Analyse critique et éthique
- Limites, risques, faux négatifs
- Place du radiologue, biais de données
